<a href="https://colab.research.google.com/github/ViniciusTavaresSousa/GAN/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from skimage.util import random_noise

#**Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Base de Dados**

In [ ]:
# Carrega a base de dados MNIST
(imagens_treinamento, _), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
# Reformata as imagens para a forma (número de imagens, 28, 28, 1) e normaliza
imagens_treinamento = imagens_treinamento.reshape(imagens_treinamento.shape[0], 28, 28, 1).astype('float32')
imagens_treinamento = (imagens_treinamento - 127.5) / 127.5

In [ ]:
# Seleciona as primeiras 10.000 imagens
imagens_treinamento = imagens_treinamento[:10000]

In [ ]:
# Configurações do dataset
buffer_size = 10000
batch_size = 256

In [ ]:
# Cria o dataset a partir das imagens, embaralha e agrupa em lotes
base_de_dados = tf.data.Dataset.from_tensor_slices(imagens_treinamento).shuffle(buffer_size).batch(batch_size)

#**GAN**

In [ ]:
# Definição das formas de entrada para o Gerador e Discriminador
input_shape_gerador = (150,)
input_shape_discriminador = [28, 28, 1]

##**Gerador**

In [ ]:
# Função para criar o modelo do Gerador
def criaGerador():
    entrada = tf.keras.Input(shape=input_shape_gerador)

    # Camadas do Gerador
    x = layers.Dense(7 * 7 * 256)(entrada)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Reshape((7, 7, 256))(x)

    x = layers.Conv2DTranspose(128, (5, 5), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    saida = layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='tanh')(x)

    modelo = tf.keras.Model(entrada, saida, name="Gerador")
    modelo.summary()  # Exibe a estrutura do modelo

    return modelo

In [ ]:
gerador = criaGerador()

Model: "Gerador"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 12544)               │       1,894,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 12544)               │          50,176 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 7, 7, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_6 (Conv2DTranspose) │ (None, 7, 7, 128)           │         819,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_7 (Conv2DTranspose) │ (None, 14, 14, 64)          │         204,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_8 (Conv2DTranspose) │ (None, 28, 28, 1)           │           1,601 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,970,881 (11.33 MB)

 Trainable params: 2,945,409 (11.24 MB)

 Non-trainable params: 25,472 (99.50 KB)

##**Discriminador**

In [ ]:
# Função para criar o modelo do Discriminador
def criaDiscriminador():
    entrada = tf.keras.Input(shape=input_shape_discriminador)

    # Camadas do Discriminador
    x = layers.Conv2D(16, (5, 5), strides=(2, 2), padding='same')(entrada)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)

    saida = layers.Dense(1)(x)

    modelo = tf.keras.Model(entrada, saida, name="Discriminador")
    modelo.summary()  # Exibe a estrutura do modelo

    return modelo

In [ ]:
discriminador = criaDiscriminador()

Model: "Discriminador"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 14, 14, 16)          │             416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_15 (LeakyReLU)           │ (None, 14, 14, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 14, 14, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 7, 7, 32)            │          12,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_16 (LeakyReLU)           │ (None, 7, 7, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 7, 7, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 4, 4, 64)            │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_17 (LeakyReLU)           │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │           1,025 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,537 (256.00 KB)

 Trainable params: 65,537 (256.00 KB)

 Non-trainable params: 0 (0.00 B)

##**Função de Perda Wasserstein**

In [ ]:
# Função de perda para o Discriminador
def perdaDiscriminador(saida_real, saida_falsa, gradient_penalty):
    c_lambda = 10
    perda_discriminador = tf.math.reduce_mean(saida_falsa) - tf.math.reduce_mean(saida_real) + c_lambda * gradient_penalty
    return perda_discriminador

In [ ]:
# Função de perda para o Gerador
def perdaGerador(saida_falsa):
    return -1. * tf.reduce_mean(saida_falsa)

##**Gradient Penalty**

In [ ]:
# Função para calcular o Gradient Penalty
@tf.function
def gradient_penalty(real, falso, epsilon):
    imagens_interpoladas = real * epsilon + falso * (1 - epsilon)
    with tf.GradientTape() as tape:
        tape.watch(imagens_interpoladas)
        scores = discriminador(imagens_interpoladas)
    gradient = tape.gradient(scores, imagens_interpoladas)[0]
    gradient_norm = tf.norm(gradient)
    gp = tf.math.reduce_mean((gradient_norm - 1)**2)
    return gp

In [ ]:
# Otimizadores do Gerador e Discriminador
gerador_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0, beta_2=0.9)
discriminador_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0, beta_2=0.9)

In [ ]:
# Diretório para checkpoints do treinamento
checkpoint_dir = "/content/drive/MyDrive/GAN/ckpt"
checkpoint_prefixo = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(
    gerador_optimizer=gerador_optimizer,
    discrimanator_optimizer=discriminador_optimizer,
    gerador=gerador,
    discriminator=discriminador)

##**Treinamento**

In [ ]:
epocas = 200 # Número de épocas de treinamento
num_de_exemplos_gerados = 1 # Número de exemplos a gerar

In [ ]:
# Diretório para armazenar os logs do TensorBoard
log_dir = "/content/drive/MyDrive/GAN/logs/gan/" + str(batch_size) + "_" + str(epocas)
summary_writer = tf.summary.create_file_writer(log_dir)

In [ ]:
seed = tf.random.normal([num_de_exemplos_gerados, input_shape_gerador[0]])

In [ ]:
# Função para treinar o modelo
def passos_treinamento(imagens, epoca):
    ruido = tf.random.normal([batch_size, input_shape_gerador[0]])
    discriminador_etapas_extras = 5

    # Treina o Discriminador
    for i in range(discriminador_etapas_extras):
        with tf.GradientTape() as d_tape:
            imagens_geradas = gerador(ruido, training=True)
            real_saida = discriminador(imagens, training=True)
            falsa_saida = discriminador(imagens_geradas, training=True)
            epsilon = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
            gp = gradient_penalty(imagens, imagens_geradas, epsilon)
            perda_discriminador = perdaDiscriminador(real_saida, falsa_saida, gp)

        discriminador_gradients = d_tape.gradient(perda_discriminador, discriminador.trainable_variables)
        discriminador_optimizer.apply_gradients(zip(discriminador_gradients, discriminador.trainable_variables))

    # Treina o Gerador
    with tf.GradientTape() as g_tape:
        imagens_geradas = gerador(ruido, training=True)
        falsa_saida = discriminador(imagens_geradas, training=True)
        perda_gerador = perdaGerador(falsa_saida)

    gerador_gradients = g_tape.gradient(perda_gerador, gerador.trainable_variables)
    gerador_optimizer.apply_gradients(zip(gerador_gradients, gerador.trainable_variables))

    # Registra as perdas no TensorBoard
    with summary_writer.as_default():
        tf.summary.scalar('Perda do Gerador', perda_gerador, step=epoca)
        tf.summary.scalar('Perda do Discriminador', perda_discriminador, step=epoca)


In [ ]:
# Função para gerar e salvar imagens durante o treinamento
def gerar_e_salvar_imagens(modelo, epoca, entrada_teste):
    predicoes = modelo(entrada_teste, training=False)

    for i in range(predicoes.shape[0]):
        imagem = np.squeeze(predicoes[i].numpy())
        imagem = (imagem + 1) * 127.5  # Reconverte a imagem para o intervalo [0, 255]
        imagem = np.clip(imagem, 0, 255)  # Garante que os valores estão entre 0 e 255
        plt.imsave(f"/content/drive/MyDrive/GAN/imagens_geradas/imagem_{epoca:04d}_{i}.png", imagem, cmap='gray')

    print(f'Imagens salvas para a época {epoca}')

In [ ]:
# Função para treinar o modelo
def treinar(base_de_dados, epocas):
    for epoca in range(epocas):
        inicio = time.time()
        for lote_de_imagens in base_de_dados:
            if len(lote_de_imagens) == batch_size:
                passos_treinamento(lote_de_imagens, epoca)

        gerar_e_salvar_imagens(gerador, epoca + 1, seed)

        # Salva o checkpoint a cada 15 épocas
        if (epoca + 1) % 15 == 0:
            checkpoint.save(file_prefix=checkpoint_prefixo)

    print(f"Tempo para a época {epoca + 1} em {time.time() - inicio:.2f} segundos")

    gerar_e_salvar_imagens(gerador, epoca + 1, seed)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/GAN/logs/gan --port 6007

In [ ]:
# Inicia o treinamento
treinar(base_de_dados, epocas)

In [ ]:
# Salvar o modelo completo no formato nativo .keras
gerador.save("/content/drive/MyDrive/GAN/gerador.keras")

##**Teste**

In [ ]:
# Carrega o modelo completo
gerador = tf.keras.models.load_model("/content/drive/MyDrive/GAN/gerador.keras")

In [ ]:
# Gerar e exibir 16 imagens com o gerador
num_exemplos = 16
seed = tf.random.normal([num_exemplos, input_shape_gerador[0]])
teste = gerador(seed, training=False)

fig, axes = plt.subplots(4, 4, figsize=(8, 8))

for i, ax in enumerate(axes.flat):
    ax.imshow(teste[i, :, :, 0], cmap='gray')  # Exibir cada imagem gerada
    ax.axis("off")  # Remover os eixos
    ax.set_title(f"Exemplo {i+1}", fontsize=10)

plt.tight_layout()
plt.show()